# Imputing Missing Data
This notebook is used to present the data imputation

In [ ]:
# For Deepnote to be able to use the custom libraries in the parent ../lib folder
import sys
sys.path.append('..')

In [ ]:
import pandas as pd
from lib.read_data import read_and_join_output_file
from lib.impute import create_transformation_pipelines, train_test_group_time_split
from lib.viz import draw_missing_data_chart
from sklearn.decomposition import PCA
from sklearn import set_config

/shared-libs/python3.9/py/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


First we load and join all the datasets resulting from the ETL process and initialize some variables.

We have two potential targets for supervised and unsupervised learning:
* `GSE_GWE` - The Ground Surface Elevation to Groundwater Water Elevation - Depth to groundwater elevation in feet below ground surface
* `SHORTAGE_COUNT` -  The number of reported well shortages

Since we have a time series datasets, the objective is to predict these values based on the historical data. For example we want to predict the 2022 count of well shortages (`SHORTAGE_COUNT`) per Township-Range based on all past data, including the past well shortages count. We thus do not split these target variables yet from the data.

In [ ]:
RANDOM_SEED = 42
df = read_and_join_output_file()

## Missing Data

In [ ]:
df.sample(5)

AttributeError: 'tuple' object has no attribute 'sample'

Let's look at the features with missing data.

In [ ]:
draw_missing_data_chart(df)

alt.LayerChart(...)

### Data Missing for Specific Years
Data were collected from the years 2014 to 2021 but some datasets only have data for specific years, when surveys were done/published. For example:
* Soils survey only has data for 2016
* Vegetations only has data for 2019
* Crops only has data for the the years 2014, 2016 and 2018
* Population density is available only for the years 2014 - 2020
* The reservoir water `PCT_OF_CAPACITY` is available only for the years 2018 - 2020

In [ ]:
crops_columns = [col for col in df if col.startswith('CROP_')]
crops_df = df[crops_columns].dropna()
soils_columns = [col for col in df if col.startswith('SOIL_')]
soils_df = df[soils_columns].dropna()
print(f"Years present in the Soils dataset {list(crops_df.index.unique(level='YEAR'))}")
print(f"Years present in the Crops dataset {list(soils_df.index.unique(level='YEAR'))}")

Years present in the Soils dataset ['2014', '2016', '2018']
Years present in the Crops dataset ['2016']


### Data Missing for Specific Township-Ranges
The Well Completion Reports dataset has data for all the years but have missing data for some specific Township-Ranges. Typically, if no wells were drilled in a specific Township-Range during the 2014-2020 period, then there is no data for that Township-Range for any of the following features:
* `TOTALDRILLDEPTH_AVG`
* `WELLYIELD_AVG`
* `STATICWATERLEVEL_AVG`
* `TOPOFPERFORATEDINTERVAL_AVG`
* `BOTTOMOFPERFORATEDINTERVAL_AVG`
* `GROUNDSURFACEELEVATION_AVG`
* `TOTALCOMPLETEDDEPTH_AVG`

Wells can also be reported with incomplete data, which means that some of the above features data could be missing for some Township-Ranges, even if wells were reported in those Township-Range.

In [ ]:
all_township_ranges = set(df.index.unique(level="TOWNSHIP_RANGE"))
wells_columns = [col for col in df if col.endswith('_AVG') or col == "TOWNSHIP_RANGE"]
wells_df = df[wells_columns].dropna()
missing_township_ranges = all_township_ranges - set(wells_df.index.unique(level="TOWNSHIP_RANGE"))
print(f"There are {len(missing_township_ranges)} out of {len(all_township_ranges)} Township-Ranges with missing well completion report data: {missing_township_ranges}")

There are 169 out of 478 Township-Ranges with missing well completion report data: {'T08S R12E', 'T05S R06E', 'T08N R08E', 'T10S R08E', 'T12S R10E', 'T22S R29E', 'T11N R22W', 'T04S R15E', 'T12N R24W', 'T03N R03E', 'T28S R19E', 'T08S R11E', 'T10N R23W', 'T06S R07E', 'T25S R26E', 'T13S R11E', 'T22S R21E', 'T12N R22W', 'T23S R20E', 'T23S R18E', 'T28S R28E', 'T28S R24E', 'T06S R15E', 'T29S R23E', 'T02N R11E', 'T20S R14E', 'T12N R19W', 'T11N R19W', 'T23S R22E', 'T32S R26E', 'T15S R11E', 'T15S R10E', 'T12S R13E', 'T31S R24E', 'T14S R15E', 'T10S R19E', 'T27S R20E', 'T05N R05E', 'T30S R23E', 'T04N R04E', 'T19S R16E', 'T22S R23E', 'T06N R10E', 'T10S R13E', 'T08S R18E', 'T22S R16E', 'T12N R18W', 'T11S R23E', 'T31S R23E', 'T03S R13E', 'T12S R09E', 'T18S R14E', 'T11S R11E', 'T07N R06E', 'T30S R21E', 'T09S R11E', 'T22S R19E', 'T11N R24W', 'T10N R21W', 'T01S R04E', 'T20S R28E', 'T23S R21E', 'T32S R30E', 'T11N R17W', 'T14S R25E', 'T24S R20E', 'T03N R05E', 'T30S R22E', 'T30S R20E', 'T10N R22W', 'T06S 

## Train-Test Split
The dataset is a time series dataset. In order to do a train-test split with use the below custom strategy:
1. To generate training and test time series we split the data by using a group split strategy base on Township-Ranges. 80% of the Township-Ranges time series are put in the training set and 20% are put in the test set.
2. We split both the training and test sets based on the year, in order to split the X and y values. In both datasets, the X values will be the data for the years 2014-2020 and the y values will be the 2021 data.


In [ ]:
X_train, X_test, y_train, y_test = train_test_group_time_split(df, index=["TOWNSHIP_RANGE", "YEAR"], group="TOWNSHIP_RANGE", random_seed=RANDOM_SEED)

Let's look at 2 examples of the training and test sets.
### The Training Set

In [ ]:
X_train.head(14)

CROP_C   CROP_C6  CROP_D10  CROP_D12  CROP_D13  CROP_D14  \
TOWNSHIP_RANGE YEAR                                                             
T01N R03E      2014     0.0  0.001229  0.000626  0.000435  0.023414   0.00012   
               2015     NaN       NaN       NaN       NaN       NaN       NaN   
               2016     0.0  0.000711  0.000968  0.000422  0.022713   0.00000   
               2017     NaN       NaN       NaN       NaN       NaN       NaN   
               2018     0.0  0.000797  0.000968  0.006218  0.021478   0.00000   
               2019     NaN       NaN       NaN       NaN       NaN       NaN   
               2020     NaN       NaN       NaN       NaN       NaN       NaN   
T01N R04E      2014     0.0  0.000000  0.000000  0.000000  0.000000   0.00000   
               2015     NaN       NaN       NaN       NaN       NaN       NaN   
               2016     0.0  0.000000  0.000000  0.000000  0.000000   0.00000   
               2017     NaN       NaN       NaN       NaN       NaN       NaN   
               2018     0.0  0.000000  0.000000  0.000000  0.000000   0.00000   
               2019     NaN       NaN       NaN       NaN       NaN       NaN   
               2020     NaN       NaN       NaN       NaN       NaN       NaN   

                     CROP_D15  CROP_D16   CROP_D3   CROP_D5  ...  \
TOWNSHIP_RANGE YEAR                                          ...   
T01N R03E      2014  0.000221  0.005375  0.027307  0.017239  ...   
               2015       NaN       NaN       NaN       NaN  ...   
               2016  0.000214  0.003037  0.032867  0.015746  ...   
               2017       NaN       NaN       NaN       NaN  ...   
               2018  0.000000  0.001684  0.047618  0.010151  ...   
               2019       NaN       NaN       NaN       NaN  ...   
               2020       NaN       NaN       NaN       NaN  ...   
T01N R04E      2014  0.000000  0.000000  0.000000  0.000000  ...   
               2015       NaN       NaN       NaN       NaN  ...   
               2016  0.000000  0.000000  0.000000  0.000000  ...   
               2017       NaN       NaN       NaN       NaN  ...   
               2018  0.000000  0.000000  0.000000  0.000000  ...   
               2019       NaN       NaN       NaN       NaN  ...   
               2020       NaN       NaN       NaN       NaN  ...   

                     GROUNDSURFACEELEVATION_AVG  STATICWATERLEVEL_AVG  \
TOWNSHIP_RANGE YEAR                                                     
T01N R03E      2014                   10.854000             44.500000   
               2015                    7.765000             30.000000   
               2016                   11.156667             26.833333   
               2017                   10.805000             37.000000   
               2018                    9.071429             41.400000   
               2019                    9.046250             35.666667   
               2020                    8.111250             36.666667   
T01N R04E      2014                         NaN                   NaN   
               2015                         NaN                   NaN   
               2016                   -2.320000              8.000000   
               2017                         NaN                   NaN   
               2018                         NaN                   NaN   
               2019                         NaN                   NaN   
               2020                         NaN                   NaN   

                     TOPOFPERFORATEDINTERVAL_AVG  TOTALDRILLDEPTH_AVG  \
TOWNSHIP_RANGE YEAR                                                     
T01N R03E      2014                   150.000000              308.000   
               2015                   180.000000              300.000   
               2016                   233.500000              360.000   
               2017                   195.000000                  NaN   
               2018         

In [ ]:
y_train.head(2)

,,CROP_C,CROP_C6,CROP_D10,CROP_D12,CROP_D13,CROP_D14,CROP_D15,CROP_D16,CROP_D3,CROP_D5,...,GROUNDSURFACEELEVATION_AVG,STATICWATERLEVEL_AVG,TOPOFPERFORATEDINTERVAL_AVG,TOTALDRILLDEPTH_AVG,TOTALCOMPLETEDDEPTH_AVG,WELLYIELD_AVG,WELL_COUNT_AGRICULTURE,WELL_COUNT_DOMESTIC,WELL_COUNT_INDUSTRIAL,WELL_COUNT_PUBLIC
TOWNSHIP_RANGE,YEAR,,,,,,,,,,,,,,,,,,,,,
T01N R03E,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.88625,25.142857,227.833333,362.75,309.0,337.4,2.0,6.0,0.0,0.0
T01N R04E,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0


### The Test Set

In [ ]:
X_test.head(14)

CROP_C   CROP_C6  CROP_D10  CROP_D12  CROP_D13  CROP_D14  \
TOWNSHIP_RANGE YEAR                                                             
T01N R02E      2014     0.0  0.005554  0.000176  0.000000  0.008879  0.003796   
               2015     NaN       NaN       NaN       NaN       NaN       NaN   
               2016     0.0  0.004971  0.001729  0.000000  0.005217  0.003099   
               2017     NaN       NaN       NaN       NaN       NaN       NaN   
               2018     0.0  0.005110  0.001337  0.000000  0.002000  0.003084   
               2019     NaN       NaN       NaN       NaN       NaN       NaN   
               2020     NaN       NaN       NaN       NaN       NaN       NaN   
T01N R11E      2014     0.0  0.000000  0.000000  0.027729  0.047432  0.000000   
               2015     NaN       NaN       NaN       NaN       NaN       NaN   
               2016     0.0  0.000000  0.000000  0.044833  0.051287  0.000000   
               2017     NaN       NaN       NaN       NaN       NaN       NaN   
               2018     0.0  0.000000  0.000000  0.029586  0.064621  0.000000   
               2019     NaN       NaN       NaN       NaN       NaN       NaN   
               2020     NaN       NaN       NaN       NaN       NaN       NaN   

                     CROP_D15  CROP_D16   CROP_D3   CROP_D5  ...  \
TOWNSHIP_RANGE YEAR                                          ...   
T01N R02E      2014  0.000015  0.002058  0.018790  0.010297  ...   
               2015       NaN       NaN       NaN       NaN  ...   
               2016  0.000704  0.000792  0.014696  0.008706  ...   
               2017       NaN       NaN       NaN       NaN  ...   
               2018  0.000690  0.000612  0.013697  0.005494  ...   
               2019       NaN       NaN       NaN       NaN  ...   
               2020       NaN       NaN       NaN       NaN  ...   
T01N R11E      2014  0.000000  0.000000  0.000000  0.000000  ...   
               2015       NaN       NaN       NaN       NaN  ...   
               2016  0.000000  0.000000  0.000000  0.000000  ...   
               2017       NaN       NaN       NaN       NaN  ...   
               2018  0.000000  0.000000  0.000000  0.000000  ...   
               2019       NaN       NaN       NaN       NaN  ...   
               2020       NaN       NaN       NaN       NaN  ...   

                     GROUNDSURFACEELEVATION_AVG  STATICWATERLEVEL_AVG  \
TOWNSHIP_RANGE YEAR                                                     
T01N R02E      2014                   56.905000             25.000000   
               2015                         NaN                   NaN   
               2016                   20.280000             44.000000   
               2017                   41.086667             31.000000   
               2018                   69.050000             75.500000   
               2019                   85.320000             35.000000   
               2020                   40.662000             47.200000   
T01N R11E      2014                  167.573333             29.666667   
               2015                  167.467500             43.000000   
               2016                  103.700000            121.000000   
               2017                         NaN                   NaN   
               2018                         NaN                   NaN   
               2019                         NaN                   NaN   
               2020                         NaN                   NaN   

                     TOPOFPERFORATEDINTERVAL_AVG  TOTALDRILLDEPTH_AVG  \
TOWNSHIP_RANGE YEAR                                                     
T01N R02E      2014                    80.000000                  NaN   
               2015                          NaN                  NaN   
               2016                   130.000000                  NaN   
               2017                   126.666667                210.0   
               2018         

In [ ]:
y_test.head(2)

,,CROP_C,CROP_C6,CROP_D10,CROP_D12,CROP_D13,CROP_D14,CROP_D15,CROP_D16,CROP_D3,CROP_D5,...,GROUNDSURFACEELEVATION_AVG,STATICWATERLEVEL_AVG,TOPOFPERFORATEDINTERVAL_AVG,TOTALDRILLDEPTH_AVG,TOTALCOMPLETEDDEPTH_AVG,WELLYIELD_AVG,WELL_COUNT_AGRICULTURE,WELL_COUNT_DOMESTIC,WELL_COUNT_INDUSTRIAL,WELL_COUNT_PUBLIC
TOWNSHIP_RANGE,YEAR,,,,,,,,,,,,,,,,,,,,,
T01N R02E,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
T01N R11E,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0


## Data Imputation
### Imputation Strategies
To impute the missing data we will use the following strategies
1. We assume little year-to-year variation in Crops, Soils and Vegetation. The missing Crops data will thus be imputed from the previous year (e.g. the 2015 data will be set as the 2014 data). For the Soils and Vegetation where we only have data for 1 year, the missing data will all be imputed from the available year.
2. The 2021 population density data will be estimated based on the 2020 population density and the 2019-2020 trend.
3. For the pre-2018 missing reservoir water `PCT_OF_CAPACITY` data, as California was affected by sever droughts during those years, we will impute missing data by taking the **minimum** `PCT_OF_CAPACITY` for that Township-Range in the post 2018 data.
4. For the well completion reports' features with missing we will use 2 distinct strategies:
    * For the  `GROUNDSURFACEELEVATION_AVG` feature we will use the median values over all the years for that Township-Range. For Township-Ranges with no data at all for any of the 2014-2020 years, we will use the median value over all Township-Ranges.
    * For the other features they will be set to 0, since these are well measurements and missing data are mainly due to no wells being drilled in that Township-Range and year.

In [ ]:
impute_pipeline, columns = create_transformation_pipelines(X_train)
X_train_impute = impute_pipeline.fit_transform(X_train)
X_test_impute = impute_pipeline.transform(X_test)

We combine the imputed training and test datasets into one dataset to visualize the results.

In [ ]:
set_config(display="diagram")
display(impute_pipeline)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder=MinMaxScaler(),
                                   transformers=[('wcr',
                                                  Pipeline(steps=[('imputer',
                                                                   PandasSimpleImputer(fill_value=0,
                                                                                       strategy='constant')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['TOTALDRILLDEPTH_AVG',
                                                   'WELLYIELD_AVG',
                                                   'STATICWATERLEVEL_AVG',
                                                   'TOPOFPERFORATEDINTERVAL_AVG',
                                                   'BOTTOMOFPERFORATEDINTERVAL_AVG',
                                                   'TOTALCOMPLETE...
                                                                   GroupImputer(aggregation_func='min',
                                                                                group_by_cols=['TOWNSHIP_RANGE'],
                                                                                impute_for_col='PCT_OF_CAPACITY')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['PCT_OF_CAPACITY']),
                                                 ('gse',
                                                  Pipeline(steps=[('imputer',
                                                                   GroupImputer(aggregation_func='median',
                                                                                group_by_cols=['TOWNSHIP_RANGE'],
                                                                                impute_for_col='GROUNDSURFACEELEVATION_AVG')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['GROUNDSURFACEELEVATION_AVG'])]))])

In [ ]:
X_train_impute_df = pd.DataFrame(X_train_impute, index=X_train.index, columns=columns)
X_test_impute_df = pd.DataFrame(X_test_impute, index=X_test.index, columns=columns)

### The Training Set

In [ ]:
X_train_impute_df.head(14)

TOTALDRILLDEPTH_AVG  WELLYIELD_AVG  STATICWATERLEVEL_AVG  \
TOWNSHIP_RANGE YEAR                                                             
T01N R03E      2014             0.097778       0.018246              0.037145   
               2015             0.095238       0.021053              0.025042   
               2016             0.114286       0.007916              0.022398   
               2017             0.000000       0.013684              0.030885   
               2018             0.083873       0.002474              0.034558   
               2019             0.078492       0.000000              0.029772   
               2020             0.074325       0.015263              0.030607   
T01N R04E      2014             0.000000       0.000000              0.000000   
               2015             0.000000       0.000000              0.000000   
               2016             0.000000       0.000000              0.006678   
               2017             0.000000       0.000000              0.000000   
               2018             0.000000       0.000000              0.000000   
               2019             0.000000       0.000000              0.000000   
               2020             0.000000       0.000000              0.000000   

                     TOPOFPERFORATEDINTERVAL_AVG  \
TOWNSHIP_RANGE YEAR                                
T01N R03E      2014                     0.098039   
               2015                     0.117647   
               2016                     0.152614   
               2017                     0.127451   
               2018                     0.148257   
               2019                     0.140359   
               2020                     0.127859   
T01N R04E      2014                     0.000000   
               2015                     0.000000   
               2016                     0.037908   
               2017                     0.000000   
               2018                     0.000000   
               2019                     0.000000   
               2020                     0.000000   

                     BOTTOMOFPERFORATEDINTERVAL_AVG  TOTALCOMPLETEDDEPTH_AVG  \
TOWNSHIP_RANGE YEAR                                                            
T01N R03E      2014                        0.111111                 0.105856   
               2015                        0.080460                 0.079848   
               2016                        0.103768                 0.104880   
               2017                        0.082375                 0.081749   
               2018                        0.093934                 0.107605   
               2019                        0.094732                 0.094011   
               2020                        0.089320                 0.089021   
T01N R04E      2014                        0.000000                 0.000000   
               2015                        0.000000                 0.000000   
               2016                        0.026054                 0.030418   
               2017                        0.000000                 0.000000   
               2018                        0.000000                 0.000000   
               2019                        0.000000                 0.000000   
               2020                        0.000000                 0.000000   

                     VEGETATION_BLUE_OAK-GRAY_PINE  \
TOWNSHIP_RANGE YEAR                                  
T01N R03E      2014                       0.000037   
               2015                       0.000037   
               2016                       0.000037   
               2017                       0.000037   
               2018                       0.000037   
               2019                       0.000037   
               2020                       0.000037   
T01N R04E      2014                       0.000236   
               2015                       0.000236   
               2

### The Test Set

In [ ]:
X_test_impute_df.head(14)

TOTALDRILLDEPTH_AVG  WELLYIELD_AVG  STATICWATERLEVEL_AVG  \
TOWNSHIP_RANGE YEAR                                                             
T01N R02E      2014             0.000000       0.000632              0.020868   
               2015             0.000000       0.000000              0.000000   
               2016             0.000000       0.004211              0.036728   
               2017             0.066667       0.008281              0.025876   
               2018             0.053651       0.000842              0.063022   
               2019             0.117460       0.000000              0.029215   
               2020             0.057778       0.008421              0.039399   
T01N R11E      2014             0.000000       0.005684              0.024763   
               2015             0.000000       0.004632              0.035893   
               2016             0.000000       0.000000              0.101002   
               2017             0.000000       0.000000              0.000000   
               2018             0.000000       0.000000              0.000000   
               2019             0.000000       0.000000              0.000000   
               2020             0.000000       0.000000              0.000000   

                     TOPOFPERFORATEDINTERVAL_AVG  \
TOWNSHIP_RANGE YEAR                                
T01N R02E      2014                     0.052288   
               2015                     0.000000   
               2016                     0.084967   
               2017                     0.082789   
               2018                     0.077124   
               2019                     0.039216   
               2020                     0.072712   
T01N R11E      2014                     0.000000   
               2015                     0.031046   
               2016                     0.133987   
               2017                     0.000000   
               2018                     0.000000   
               2019                     0.000000   
               2020                     0.000000   

                     BOTTOMOFPERFORATEDINTERVAL_AVG  TOTALCOMPLETEDDEPTH_AVG  \
TOWNSHIP_RANGE YEAR                                                            
T01N R02E      2014                        0.075670                 0.128327   
               2015                        0.000000                 0.000000   
               2016                        0.057471                 0.057034   
               2017                        0.063857                 0.074778   
               2018                        0.052874                 0.064259   
               2019                        0.137931                 0.140684   
               2020                        0.055556                 0.068821   
T01N R11E      2014                        0.093614                 0.000000   
               2015                        0.088602                 0.079848   
               2016                        0.086207                 0.091255   
               2017                        0.000000                 0.000000   
               2018                        0.000000                 0.000000   
               2019                        0.000000                 0.000000   
               2020                        0.000000                 0.000000   

                     VEGETATION_BLUE_OAK-GRAY_PINE  \
TOWNSHIP_RANGE YEAR                                  
T01N R02E      2014                       0.010798   
               2015                       0.010798   
               2016                       0.010798   
               2017                       0.010798   
               2018                       0.010798   
               2019                       0.010798   
               2020                       0.010798   
T01N R11E      2014                       0.158617   
               2015                       0.158617   
               2

In [ ]:
draw_missing_data_chart(X_train_impute_df)

alt.LayerChart(...)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b042e2da-6536-449d-95b8-d85fa08825de' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>